In [24]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [26]:
tf.keras.backend.clear_session()

In [27]:
subject = 'Paraíba - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [28]:
data = pd.read_csv('2003_mo_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Consumo de Cimento (t)
0,2003-1,8.315958,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,118.926160,0.182304,1.190368,948.129859,1343.725535,22.497513,19.371613,84.002026,4.942143,0.656510,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,38.426
1,2003-2,8.309895,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,117.724891,0.185909,1.190430,948.130898,1343.725535,22.497092,19.371751,84.004982,4.939870,0.656949,3.021010e+07,1.632950e+06,7.293438,2.737516e+07,31.672
2,2003-3,8.303831,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,118.501965,0.189593,1.190466,948.131504,1343.725535,22.496846,19.371831,84.006706,4.938543,0.657388,3.024728e+07,1.634566e+06,7.295708,2.738689e+07,28.775
3,2003-4,8.297767,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,119.390679,0.193353,1.190470,948.131569,1343.725535,22.496820,19.371840,84.006891,4.938401,0.657827,3.028445e+07,1.636182e+06,7.297978,2.739862e+07,28.571
4,2003-5,8.291704,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,121.517541,0.197186,1.212656,948.032498,1337.766989,22.616890,19.394367,83.575985,4.915516,0.658266,3.032163e+07,1.637798e+06,7.300247,2.741035e+07,31.613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.165653,0.679715,1.201229,949.160060,311.814483,21.701965,18.705257,84.493745,3.040558,NaN,NaN,NaN,NaN,NaN,95.860
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.457511,0.678961,1.121888,948.762712,352.255801,22.148149,18.829599,83.138667,2.865656,NaN,NaN,NaN,NaN,NaN,90.790
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.809622,0.677475,1.060573,948.492322,444.902027,22.420311,18.931075,82.430549,2.726301,NaN,NaN,NaN,NaN,NaN,94.588
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.980478,0.676277,1.031293,948.261053,613.174462,22.580160,19.009295,82.117924,2.410941,NaN,NaN,NaN,NaN,NaN,89.777


In [29]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,0.010092,-0.251051,0.921828,-0.130628,0.076478,0.200221,1.422757,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,0.010389,-0.249966,0.921828,-0.131105,0.076697,0.201119,1.417180,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,0.010563,-0.249333,0.921828,-0.131383,0.076825,0.201643,1.413928,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,0.010581,-0.249265,0.921828,-0.131413,0.076839,0.201699,1.413579,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,0.116637,-0.352729,0.887247,0.004435,0.112697,0.070805,1.357455,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,-1.873518,1.126091,-0.809266,-0.476338,-1.806568,-1.005605,0.126603,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,-1.840689,0.657168,-0.895246,-0.026902,-1.614287,-1.327471,0.068512,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,-1.364317,0.418383,-1.101157,0.487147,-1.299644,-1.634129,-0.033523,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,-0.105041,0.157363,-1.377932,0.777098,-0.647516,-1.426909,-0.449403,1.213827,1.057806,-0.147797,0.624852,0.688355


In [30]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Paraíba - Consumo de Cimento (t), Length: 240, dtype: float64

In [31]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
0,-0.800943,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-2.092830,-2.424081,0.010092,-0.251051,0.921828,-0.130628,0.076478,0.200221,1.422757,-2.168844,-1.710694,-2.484291,-2.270808,-2.124240
1,-0.805366,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-2.140285,-2.392248,0.010389,-0.249966,0.921828,-0.131105,0.076697,0.201119,1.417180,-2.133132,-1.692339,-2.436666,-2.234559,-2.091568
2,-0.809789,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-2.109587,-2.359718,0.010563,-0.249333,0.921828,-0.131383,0.076825,0.201643,1.413928,-2.097421,-1.673984,-2.389041,-2.198310,-2.058896
3,-0.814212,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-2.074479,-2.326517,0.010581,-0.249265,0.921828,-0.131413,0.076839,0.201699,1.413579,-2.061709,-1.655629,-2.341417,-2.162061,-2.026224
4,-0.818635,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-1.990458,-2.292670,0.116637,-0.352729,0.887247,0.004435,0.112697,0.070805,1.357455,-2.025998,-1.637274,-2.293792,-2.125812,-1.993552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.300589,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,0.747301,0.928050,0.467517,0.380858,-0.952640,0.136753,1.143376,0.925935,-1.354693,1.431308,1.193935,-0.099354,0.924381,1.014355
158,1.292964,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,0.751028,0.933525,0.549541,0.768975,-1.198162,-0.146203,0.935577,1.052656,-1.337929,1.429336,1.195558,-0.105914,0.923232,1.010769
159,1.285339,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,0.748338,0.939401,0.215844,1.146631,-1.391707,-0.578393,0.305946,0.983698,-1.085829,1.427365,1.197181,-0.112475,0.922084,1.007184
160,1.277715,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,0.708461,0.946049,-0.345342,1.700464,-1.183820,-0.879232,-0.598461,0.529584,-0.816486,1.425393,1.198804,-0.119036,0.920935,1.003598


In [32]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      31.891
1      26.089
2      32.066
3      26.351
4      27.348
        ...  
157    58.962
158    73.459
159    59.150
160    64.833
161    63.840
Name: Paraíba - Consumo de Cimento (t), Length: 162, dtype: float64

In [33]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [34]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [35]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Paraíba - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Paraíba - value,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado
126,0.913576,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,0.690698,0.725920,-0.225666,1.056938,-0.982278,-1.147409,-0.671619,0.748504,-0.204513,0.453223,0.844953,0.848711,1.030555,1.033347
127,0.936878,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,0.689517,0.735237,-0.651839,0.768312,-0.736242,-1.102054,-1.018660,0.375547,-0.127027,0.463389,0.864872,0.830125,1.045296,1.048217
128,0.960180,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,0.699239,0.744336,-0.949203,0.428765,-0.446715,-0.754002,-1.041292,-0.053088,-0.317716,0.473555,0.884791,0.811540,1.060038,1.063087
129,0.983482,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,0.679647,0.753547,-1.028418,0.052148,-0.192869,-0.381013,-1.089776,-0.529773,-0.446748,0.483721,0.904710,0.792954,1.074779,1.077957
130,1.006783,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,0.659340,0.762358,-0.265577,-0.396379,-0.204838,-0.190645,-0.873933,-0.576166,-0.740392,0.493887,0.924630,0.774368,1.089520,1.092827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.272641,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,1.885384,0.924040,-1.873518,1.126091,-0.809266,-0.476338,-1.806568,-1.005605,0.126603,1.235988,1.087341,-0.170948,0.675345,0.728937
188,1.277530,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,1.959697,0.934231,-1.840689,0.657168,-0.895246,-0.026902,-1.614287,-1.327471,0.068512,1.228601,1.077496,-0.163231,0.658514,0.715410
189,1.282419,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,2.094602,0.957607,-1.364317,0.418383,-1.101157,0.487147,-1.299644,-1.634129,-0.033523,1.221214,1.067651,-0.155514,0.641683,0.701882
190,1.287308,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,2.203705,0.987746,-0.105041,0.157363,-1.377932,0.777098,-0.647516,-1.426909,-0.449403,1.213827,1.057806,-0.147797,0.624852,0.688355


In [36]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [37]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [38]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [47]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [48]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3183281583, 2144068827, 377174606, 2439070999, 1431568050]


Step: 0 ___________________________________________
val_loss: 51.03448486328125
winner_seed: 3183281583


Step: 1 ___________________________________________
val_loss: 52.73161315917969


Step: 2 ___________________________________________
val_loss: 45.46523666381836
winner_seed: 377174606


Step: 3 ___________________________________________
val_loss: 58.95816421508789


Step: 4 ___________________________________________
val_loss: 49.886566162109375


final_seed: 377174606


In [49]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 116ms/step - loss: 5364.7651 - val_loss: 523.2644
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 622.8831 - val_loss: 356.7586
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 392.2299 - val_loss: 784.4551
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 622.6875 - val_loss: 297.3383
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 233.7009 - val_loss: 179.7775
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 596.7379 - val_loss: 291.2780
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 391.9535 - val_loss: 278.6666
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 488.6624 - val_loss: 297.4769
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 213.4513 - val_loss: 178.3152
Epoch 10/10000
4/4 [==============================] - 0s 9ms/step -

4/4 [==============================] - 0s 9ms/step - loss: 25.0049 - val_loss: 60.6705
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 26.7276 - val_loss: 61.5472
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.7200 - val_loss: 64.9681
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 25.2770 - val_loss: 61.5157
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.7923 - val_loss: 67.9790
Epoch 164/10000
4/4 [==============================] - 0s 11ms/step - loss: 33.8700 - val_loss: 65.4403
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 37.2912 - val_loss: 60.9231
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.7346 - val_loss: 61.3302
Epoch 167/10000
4/4 [==============================] - 0s 9ms/step - loss: 25.7122 - val_loss: 64.2002
Epoch 168/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.6295 - v

4/4 [==============================] - 0s 8ms/step - loss: 23.8784 - val_loss: 62.6937
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.4209 - val_loss: 95.6308
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.1960 - val_loss: 65.6258
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.8269 - val_loss: 62.2346
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.5510 - val_loss: 59.0636
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 22.9536 - val_loss: 58.6493
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.4339 - val_loss: 66.5910
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.5035 - val_loss: 61.2990
Epoch 325/10000
4/4 [==============================] - 0s 9ms/step - loss: 25.7277 - val_loss: 60.4346
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.0496 - val

4/4 [==============================] - 0s 10ms/step - loss: 20.4298 - val_loss: 84.8528
Epoch 476/10000
4/4 [==============================] - 0s 9ms/step - loss: 20.0016 - val_loss: 57.3656
Epoch 477/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.4139 - val_loss: 62.3347
Epoch 478/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.1704 - val_loss: 57.4035
Epoch 479/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.9451 - val_loss: 60.0798
Epoch 480/10000
4/4 [==============================] - 0s 9ms/step - loss: 20.6111 - val_loss: 55.7810
Epoch 481/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.8171 - val_loss: 57.6786
Epoch 482/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.3387 - val_loss: 63.6765
Epoch 483/10000
4/4 [==============================] - 0s 9ms/step - loss: 20.4651 - val_loss: 57.6005
Epoch 484/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.0852 - v

4/4 [==============================] - 0s 10ms/step - loss: 11.5358 - val_loss: 62.8610
Epoch 634/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.2340 - val_loss: 52.5989
Epoch 635/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.9109 - val_loss: 70.7597
Epoch 636/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.7812 - val_loss: 61.4210
Epoch 637/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.4105 - val_loss: 56.2044
Epoch 638/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.7674 - val_loss: 66.4900
Epoch 639/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.4748 - val_loss: 78.5536
Epoch 640/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.3588 - val_loss: 76.9181
Epoch 641/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1264 - val_loss: 69.9266
Epoch 642/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.4991 - v

...

Epoch 950/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.8265 - val_loss: 76.4572
Epoch 951/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.9003 - val_loss: 66.0650
Epoch 952/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.1274 - val_loss: 103.1713
Epoch 953/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.9124 - val_loss: 53.6503
Epoch 954/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.0764 - val_loss: 63.5602
Epoch 955/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.1167 - val_loss: 74.4279
Epoch 956/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.9356 - val_loss: 76.1374
Epoch 957/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.3787 - val_loss: 77.9927
Epoch 958/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.5135 - val_loss: 79.7293
Epoch 959/10000
4/4 [==============================] - 0s 9ms/step - loss

4/4 [==============================] - 0s 9ms/step - loss: 7.1387 - val_loss: 60.3242
Epoch 1109/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6233 - val_loss: 73.3439
Epoch 1110/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6080 - val_loss: 63.3531
Epoch 1111/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4349 - val_loss: 65.4666
Epoch 1112/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.9970 - val_loss: 70.8646
Epoch 1113/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.4404 - val_loss: 70.2859
Epoch 1114/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8005 - val_loss: 65.4205
Epoch 1115/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5261 - val_loss: 62.7862
Epoch 1116/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4178 - val_loss: 61.2522
Epoch 1117/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0786 - val_l

4/4 [==============================] - 0s 9ms/step - loss: 8.2692 - val_loss: 64.2221
Epoch 1267/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.7463 - val_loss: 54.0923
Epoch 1268/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.6240 - val_loss: 56.5939
Epoch 1269/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.4643 - val_loss: 66.3502
Epoch 1270/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7830 - val_loss: 60.6500
Epoch 1271/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.2228 - val_loss: 53.5356
Epoch 1272/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.6245 - val_loss: 57.7069
Epoch 1273/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.0130 - val_loss: 64.1196
Epoch 1274/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.9839 - val_loss: 61.0946
Epoch 1275/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.1535 -

4/4 [==============================] - 0s 10ms/step - loss: 3.2921 - val_loss: 70.4322
Epoch 1425/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.9998 - val_loss: 73.7546
Epoch 1426/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6025 - val_loss: 66.8797
Epoch 1427/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4473 - val_loss: 66.7398
Epoch 1428/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.2101 - val_loss: 71.2967
Epoch 1429/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.8759 - val_loss: 68.3614
Epoch 1430/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.1509 - val_loss: 73.1831
Epoch 1431/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0268 - val_loss: 72.7897
Epoch 1432/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.8538 - val_loss: 71.0301
Epoch 1433/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.874

4/4 [==============================] - 0s 10ms/step - loss: 3.7458 - val_loss: 68.4897
Epoch 1583/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.9864 - val_loss: 71.5173
Epoch 1584/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.4672 - val_loss: 71.0927
Epoch 1585/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.5334 - val_loss: 70.6284
Epoch 1586/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.6655 - val_loss: 70.9228
Epoch 1587/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.8472 - val_loss: 68.2274
Epoch 1588/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.3624 - val_loss: 67.1628
Epoch 1589/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0252 - val_loss: 71.8891
Epoch 1590/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.2483 - val_loss: 68.7662
Epoch 1591/10000
4/4 [==============================] - 0s 9ms/step - loss: 3.965

In [50]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [51]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,62.417336,62.944939,63.029724,63.686661,61.924629,63.598473,66.778801,66.728119,66.740372,65.790657,62.609169,60.990097,61.34066,62.398529,63.9245,68.149353,67.754547,66.463547,64.138428,63.582314,64.241753,65.560623,67.398666,65.918762,63.276535,67.760002,64.257202,62.40448,61.399605,64.211914
Target,61.721,69.24,70.048,67.172,64.22,63.021,62.639,53.839,56.652,53.408,50.543,64.699,60.219,64.217,57.591,64.307,61.971,57.48,68.053,55.231,53.723,57.457,62.755,48.148,62.19,62.219,61.329,81.988,79.341,76.646
Error,0.696335,6.295059,7.018272,3.485336,2.295372,0.577473,4.139801,12.889118,10.088371,12.382656,12.06617,3.7089,1.121658,1.818474,6.3335,3.842354,5.783546,8.983547,3.914574,8.351315,10.518753,8.103622,4.643665,17.770763,1.086536,5.541,2.928204,19.583519,17.941399,12.43409


In [52]:
display(mae)
display(mape)

7.2114463

0.118235916

In [53]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [54]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[767.239]] - Target[737.202]| =  Error: [[30.036987]]; MAPE:[[0.04074458]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[780.8717]] - Target[711.152]| =  Error: [[69.71973]]; MAPE:[[0.09803773]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[383.30972]] - Target[423.713]| =  Error: [[40.40329]]; MAPE:[[0.09535532]]


[array([[30.036987]], dtype=float32),
 array([[69.71973]], dtype=float32),
 array([[40.40329]], dtype=float32)]

46.72

0.07804588